# Trabajo final de TAL

Este trabajo consiste en implementar un pipeline RAG funcional que utilize un conjunto de noticias de robos violentos en chile desde 2022 a 2025.

### Integrantes 

- Renato Atencio
- Tomas Contreras

## Funcionamiento

Para el sistema se opto por utilizar LM Studio, ya que no queriamos dejar los datos bancarios a openai o google para utilizar los modelos. Por ende se descargo esta aplicacion y se utilizo un modelo gratuito. <br> 
Esto tambien nos facilita el probar el sistema ya que no estamos limitados por la cantidad de tokens, aunque la precision de modelo no sera la misma que los modelos de paga.

- **Modelo llm**: Openai/gpt-oss-20b

Tambien se opto por utilizar ChromaDB, lo cual nos permite almacenar dmbeddings y hacer las busquedas por similitud. Considerando que tenemos 342 documentos y que se queria permitir la busqueda semantica, nos parecio una idea correcta. <br>
Para realizar estos embeddings se utilizo:

- **Modelo Embeddings**: "paraphrase-multilingual-mpnet-base-v2"

Que es un modelo bastante utilizado ya que es gratuito, local y preciso para textos en español. Este modelo separa los textos en tokens, los pasa por una red *Trasformer* que analisa el orden, contexto y las relaciones entre palabra y produce un vector de 768 dimensiones.

Finalmente para las consultas tambien se utiliza otro modelo para rerankear los resultados a la busqueda, esto debido a que el modelo es pequeño y no permite pasar mas de 3~5 textos, asi que se obtienen los x resultados semanticos y se pasan los y resultados mas relevante al llm

- **Modelo de reranking**: cross-encoder/ms-marco-MiniLM-L-6-v2

Finalmente estos vectores + metadata se dejan en chromadb, y se pueden realizar las busquedas semanticas y con filtros.

## Compilacion/Ejecucion

Para hacer las consultas al llm, se craron 2 funciones, una que permite definir un texto para la busqueda semantica y una pregunta para realizar sobre los textos obtenido, y otra que es similar pero que permite filtros.

In [ ]:
# Imports
import os
import chromadb
import pandas as pd
from openai import OpenAI
from dotenv import load_dotenv
from sentence_transformers import CrossEncoder
from sentence_transformers import SentenceTransformer


In [3]:
# Variables de entorno
load_dotenv()
csv_path = os.getenv("CSVPATH")
chroma_path = os.getenv("CHROMAPATH")
chroma_collection = os.getenv("CHROMACOLL")

In [4]:
df = pd.read_csv(csv_path)

# Pasar fecha a formato espeficio para ChromaDB
df["date"] = pd.to_datetime(df["date"], errors="coerce")
df["date"] = df["date"].dt.strftime("%Y-%m-%d")
df["date"] = df["date"].fillna("unknown")

# Esto crea una nueva columna en el df, que basicamente es la combinacion del titulo, texto, pais, medio de comunicacion, fecha de publicacion y popularidad del medio (Esto es para agregar mas datos para el embedding aparte de solo el texto)
df["rag_text"] = df.apply(
    lambda row: (
        f"Título: {row['title']}\n"
        f"Texto: {row['text']}\n"
        f"País: {row['country']}\n"
        f"Medio: {row['media_outlet']}\n"
        f"Fecha: {row['date']}\n"
        f"Polaridad: {row['polarity_label']}"
    ),
    axis=1
)

C:\Users\Renat\AppData\Local\Temp\ipykernel_10832\3585856804.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["date"] = pd.to_datetime(df["date"], errors="coerce")


In [5]:
# Modelo de embedding
model = SentenceTransformer("paraphrase-multilingual-mpnet-base-v2")

# Cliente de ChromaDB
chroma_client = chromadb.PersistentClient(path=chroma_path)

# Crear collecion en ChromaDB
collection = chroma_client.get_or_create_collection(
    name="mi_coleccion",
    metadata={"hnsw:space": "cosine"}
)

# Proceso de embedding
documents = df["rag_text"].astype(str).tolist()
embeddings = model.encode(documents, batch_size=32, normalize_embeddings=True)

# Esto genera un arreglo de metadata para cada documento, permite busquedas con filtros especificos
metadata = df[["country", "media_outlet", "date", "year", "month", "polarity_label"]].to_dict(orient="records")

# Se agregan los datos. Embeddings y metadata son del mismo largo y deberian de estar ordenados (Osea el embeddings[0] -> embedding texto 0, metadata[0] -> metadata texto 0)
collection.add(
    documents=documents,
    embeddings=embeddings,
    metadatas=metadata,
    ids=[f"id_{i}" for i in range(len(documents))]
)

print(f"Documentos insertados: {collection.count()}")

# Entonces, con esto se obtubo una db vectorizada con los embeddings y metadata, lo cual permite la busqueda semantica y por filtros

Documentos insertados: 342


In [ ]:
# Para borrar la coleccion
# chroma_client.delete_collection(name="mi_coleccion")

In [6]:
# Cliente LLM con LM Studio
llm = OpenAI(
    base_url="http://localhost:1234/v1",
    api_key="lmstudio"  # cualquier texto
)

In [ ]:
# Modelo de reranking (evalua que tan relevante es cada doc para la query)
reranker = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')

In [30]:
def rag_query_simple(query_textos, cant_res_ini, cant_res_fin, pregunta):
    # Se toma la query y se pasa a un vector de 768 num
    query_embedding = model.encode([query_textos], normalize_embeddings=True)
    
    # Sacar documentos con la query
    results = collection.query(
        query_embeddings=query_embedding.tolist(),
        n_results=cant_res_ini
    )

    # Sacar solo el texto se que uso en el embedding
    docs = results["documents"][0]

    # Se realiza un ranking de los resultados segun la busqueda
    pairs = [[query_textos, doc] for doc in docs]
    scores = reranker.predict(pairs)

    # Se obtienen solo los mejores [cant_res_fin] resultados
    ranked_indices = sorted(range(len(scores)), key=lambda i: scores[i], reverse=True)
    top_docs = [docs[i] for i in ranked_indices[:cant_res_fin]]

    # Unir los textos como contexto para el modelo
    context = "\n\n".join(top_docs)

    prompt = f"""
        Responde la pregunta usando EXCLUSIVAMENTE el siguiente contexto:

        <contexto>
        {context}
        </contexto>

        Pregunta:
        {pregunta}

        Responde de manera clara y concisa:
        """

    # Llamar al llm (Tiene que estar corriendo el server del LM Studio, con el modelo, asegurar que el modelo sea exacto: openai/gpt-oss-20b)
    response = llm.chat.completions.create(
        model="openai/gpt-oss-20b",  # reemplazar por el nombre exacto del modelo
        messages=[{"role": "user", "content": prompt}]
    )

    answer = response.choices[0].message.content
    return answer, docs

def rag_query_simple_filter(query_textos, cant_res_ini, cant_res_fin, pregunta, filter):
    query_embedding = model.encode([query_textos], normalize_embeddings=True)
    
    # Sacar documentos con la query
    results = collection.query(
        query_embeddings=query_embedding.tolist(),
        n_results=cant_res_ini,
        where=filter
    )

    # Sacar solo el texto se que uso en el embedding
    docs = results["documents"][0]

    # Se realiza un ranking de los resultados segun la busqueda
    pairs = [[query_textos, doc] for doc in docs]
    scores = reranker.predict(pairs)

    # Se obtienen solo los mejores [cant_res_fin] resultados
    ranked_indices = sorted(range(len(scores)), key=lambda i: scores[i], reverse=True)
    top_docs = [docs[i] for i in ranked_indices[:cant_res_fin]]

    # Unir los textos como contexto para el modelo
    context = "\n\n".join(top_docs)

    prompt = f"""
        Responde la pregunta usando EXCLUSIVAMENTE el siguiente contexto:

        <contexto>
        {context}
        </contexto>

        Pregunta:
        {pregunta}

        Responde de manera clara y concisa:
        """

    # Llamar al llm (Tiene que estar corriendo el server del LM Studio, con el modelo, asegurar que el modelo sea exacto: openai/gpt-oss-20b)
    response = llm.chat.completions.create(
        model="openai/gpt-oss-20b",  # reemplazar por el nombre exacto del modelo
        messages=[{"role": "user", "content": prompt}]
    )

    answer = response.choices[0].message.content
    return answer, docs

In [33]:
query = "Delincuente menor de edad es detenido nuevamente"
pregunta = "De estas noticias, en cuales se habla de un delincuente menor de edad con multiples detenciones"

respuesta, docs = rag_query_simple(query, 20, 3, pregunta)

print("Respuesta del llm:\n", respuesta)

print("Documentos Usados:")
for i in range(0, len(docs)):
    print(f"Doc {i}:\n{docs[i]}")
    print("============================")

Respuesta del llm:
 La noticia que menciona a un delincuente menor de edad con múltiples detenciones es la primera, titulada **“Menor fugado del hospital fue detenido tras cometer un nuevo robo”**, donde se habla del adolescente de 17 años J.I.G.A., imputado por dos robos y posteriormente involucrado en otro delito.
Documentos Usados:
Doc 0:
Título: Ocho horas alcanzó a estar detenido adolescente imputado por robo con cuchillo
Texto: Realizan exitosa jornada de limpieza comunitaria en el Humedal Tres Puentes Iniciativa busca instalar internet satelital en 15 escuelas rurales de Magallanes Más de 1.500 personas asistieron al primer “Chapuzón Kids” en Punta Arenas Alcalde agasajó a recolector que devolvió 27 millones de pesos que encontró en la vía pública ¡Notable! Devolvió bolso con $27 millones que encontró en la basura Se entregó adolescente involucrado en el homicidio ocurrido en la Plaza de Armas de Natales Adolescente de 17 años fue asesinado en la Plaza de Armas de Puerto Natales

In [31]:
query = "asesinato durante intento de robo"
pregunta = "De estas noticias, en cuales se menciona un asesinato y que zona de la cuidad se menciona mas"
filter = { "country": "chile" }

respuesta, docs = rag_query_simple_filter(query, 20, 3, pregunta, filter)

print("Respuesta del llm:\n", respuesta)

print("Documentos Usados:")
for i in range(0, len(docs)):
    print(f"Doc {i}:\n{docs[i]}")
    print("============================")

Respuesta del llm:
 - **Asesinato mencionado:** Solo en la segunda noticia se habla de un homicidio (la muerte de la carabinera Rita Olivares).

- **Zona más citada:** La zona que aparece con mayor frecuencia es **Quilpue, Región de Valparaíso**.
Documentos Usados:
Doc 0:
Título: Roban fusil y cargadores en ataque armado a campo de entrenamiento naval en Viña del Mar
Texto: Un robo ocurrió durante la madrugada de este jueves en el Campo de Entrenamiento Almirante Bascuñán (CEAB) de la Armada de Chile, en Viña del Mar: cinco individuos armados y encapuchados ingresaron al recinto, enfrentaron al personal de guardia y sustrajeron material institucional antes de darse a la fuga. Según información de la Primera Zona Naval, los sujetos dispararon contra la caseta de seguridad y lograron reducir a los guardias, sustrayendo un fusil y cargadores antes de escapar. “El hecho fue denunciado al Ministerio Público y a la Fiscalía Naval, iniciándose las investigaciones correspondientes”, señaló la 

## Conclusiones

Los sistemas RAG permiten el analisis de multiples textos de manera veloz pero con un costo computacional alto, donde se terminan utilizando multiples modelos de IA. 